# System creation with MMElemental

### Ligand and receptor creation 

In [1]:
from mmelemental.models import Molecule
from mm_data import mols


# Construct MMSchema molecules
receptor = Molecule.from_file(mols["alanine.pdb"])
ligand   = Molecule.from_file(mols["ibu.pdb"])

### Visualizing molecules

In [2]:
receptor

NGLWidget()

# Ligand conformation generation with QCEgnine

### MMSchema to QCSchema mol conversion

In [3]:
qc_ligand = ligand.to_data("qcschema").data

qc_ligand

NGLWidget()

### Geometry optimization

In [5]:
import qcengine

# Create qc input
opt_input = {
    "keywords": {
        "program": "rdkit"
    },
    "input_specification": {
        "driver": "gradient",
        "model": {"method": "UFF"},
    },
    "initial_molecule": qc_ligand
}

# Run compute
opt = qcengine.compute_procedure(opt_input, "geometric")

# Extract optimized molecule
qc_ligand_opt = opt.final_molecule

# Ibuprofen after optimization
qc_ligand_opt

AttributeError: 'FailedOperation' object has no attribute 'final_molecule'

# Molecular docking with strategy component: mmic_docking

### QC-ligand to MMSchema mol conversion


In [5]:
ligand_opt = Molecule.from_data(qc_ligand_opt, dtype="qcschema")

### Docking computation

In [6]:
import mmic_docking


# Construct docking input data from MMSchema molecules
dockin = {
    "molecule": {
        "ligand": ligand_opt, 
        "receptor": receptor,
    },
    "search_space": (-37.807, 5.045, -2.001, 30.131, -19.633, 37.987),
    "search_space_units": "angstrom",
    # we can force mmic_docking to run a component of our choice
    #"component": "mmic_autodock_vina", 
    
}

dockout = mmic_docking.components.DockComponent.compute(dockin)

scores, poses = dockout.scores, dockout.poses.ligand

print("Scores:")
print("=======")
print(scores)

print("\nLigand poses:")
print("=============")
poses

Scores:
[-1.2, -1.2, -1.1, -1.1, -1.1, -1.1, -1.1, -1.1, -1.0]

Ligand poses:


[Molecule(name='C13HO2', hash='c43a86e'),
 Molecule(name='C13HO2', hash='6731439'),
 Molecule(name='C13HO2', hash='c5e8733'),
 Molecule(name='C13HO2', hash='211ba9b'),
 Molecule(name='C13HO2', hash='21bdf72'),
 Molecule(name='C13HO2', hash='ba3f0f9'),
 Molecule(name='C13HO2', hash='17180ed'),
 Molecule(name='C13HO2', hash='e83742a'),
 Molecule(name='C13HO2', hash='a2c7742')]